# Using SFEPy to solve a linear elasticity problem\

 # $-\frac{\partial\sigma_{ij}}{\partial x_j} + f_i = 0$   in   $\Omega$,  $\bf{u} = 0$ on $\Gamma_1$, $u_1 = f(x)$ on $\Gamma_2$

## The stress is $\sigma_{ij} = 2\mu\epsilon_{ij}+\lambda\epsilon_{kk}\delta_{ij}$ where $\lambda$ and $\mu$ are the Lame constants

### The strain is 
$\epsilon_{ij}(\bf{u}) = \frac{1}{2}\big(\frac{\partial u_i}{\partial x_j} +\frac{\partial u_j}{\partial x_i}  \big)$ 



### while $\bf{f}$ are volume forces. 
### With this the stress tensor components can be written in the following general form: 

### $\sigma_{ij}(\bf{u})=D_{ijkl}\epsilon_{kl}(\bf{u})$ where in this case $D_{ijkl}=\mu(\delta_{ik}\delta_{jl}+\delta_{il}\delta_{jk}) +\lambda\delta_{ij}\delta_{kl}$ .

### The weak form ot the above equation can be written in the following form 

### $\int_\Omega D_{ijkl}\epsilon_{kl}(\bf{u})\epsilon(\bf{v}) +\int_\Omega f_i v_i = 0$

In [2]:
import numpy as np
from sfepy.discrete.fem import Mesh, FEDomain, Field

### Read and define the mesh

In [3]:
mesh = Mesh.from_file('../sfepy/meshes/2d/rectangle_tri.mesh')

sfepy: reading mesh (../sfepy/meshes/2d/rectangle_tri.mesh)...
sfepy:   number of vertices: 258
sfepy:   number of cells:
sfepy:     2_3: 454
sfepy: ...done in 0.01 s


### Create the domain and regions 

In [4]:
domain = FEDomain('domain', mesh)

In [5]:
min_x, max_x = domain.get_mesh_bounding_box()[:, 0]

In [6]:
eps = 1e-8 * (max_x - min_x)

In [7]:
omega = domain.create_region('Omega', 'all')

In [8]:
gamma1 = domain.create_region('Gamma1','vertices in x < %.10f' % (min_x + eps),'facet')

In [9]:
gamma2 = domain.create_region('Gamma2','vertices in x > %.10f' % (max_x - eps),'facet')

### Define the Finite element approximation

In [10]:
field = Field.from_args('fu', np.float64, 'vector', omega,approx_order=2)

In [11]:
from sfepy.discrete import (FieldVariable, Material, Integral, Function,Equation, Equations, Problem)

### Define unknown and test variables

In [12]:
u = FieldVariable('u', 'unknown', field)
v = FieldVariable('v', 'test', field, primary_var_name='u')

### Define material properties and

In [50]:
from sfepy.mechanics.matcoefs import stiffness_from_lame
m = Material('m', D=stiffness_from_lame(dim=2, lam=1.0, mu=1.0))
f = Material('f', val=[[0.0], [0.0]])

### Define quadrature order

In [14]:
integral = Integral('i', order=3)

### Define terms and build equations

In [51]:
from sfepy.terms import Term
t1 = Term.new('dw_lin_elastic(m.D, v, u)',integral, omega, m=m, v=v, u=u)
t2 = Term.new('dw_volume_lvf(f.val, v)',integral, omega, f=f, v=v)
eq = Equation('balance', t1 + t2)
eqs = Equations([eq])

### Set boundary conditions

In [52]:
from sfepy.discrete.conditions import Conditions, EssentialBC
fix_u = EssentialBC('fix_u', gamma1, {'u.all' : 0.0})


In [53]:
omega = domain.create_region('Omega', 'all')

In [91]:
def shift_u_fun(ts, coors, bc=None, problem=None, shift=0.0):
    val = shift * coors[:,1]**3
    return val

In [92]:
bc_fun = Function('shift_u_fun', shift_u_fun,
                  extra_args={'shift' : 0.01})

In [93]:
shift_u = EssentialBC('shift_u', gamma2, {'u.0' : bc_fun})

### Define linear and non-linear solvers

In [94]:
from sfepy.base.base import IndexedStruct
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton


In [95]:
ls = ScipyDirect({})
nls_status = IndexedStruct()
nls = Newton({}, lin_solver=ls, status=nls_status)

### Create a Problem instance and solve it 

In [96]:
pb = Problem('elasticity', equations=eqs)

In [97]:
pb.save_regions_as_groups('regions')

sfepy: saving regions as groups...
sfepy:   Omega
sfepy:   Gamma1
sfepy:   Gamma2
sfepy:   Omega
sfepy:   Omega
sfepy: ...done


In [98]:
pb.set_bcs(ebcs=Conditions([fix_u, shift_u]))

In [99]:
pb.set_solver(nls)

In [100]:
status = IndexedStruct()

In [101]:
vec = pb.solve(status=status)

sfepy: updating variables...
sfepy: ...done
sfepy: matrix shape: (1815, 1815)
sfepy: assembling matrix graph...
sfepy: ...done in 0.00 s
sfepy: matrix structural nonzeros: 39145 (1.19e-02% fill)
sfepy: updating variables...
sfepy: ...done
sfepy: updating materials...
sfepy:     m
sfepy:     f
sfepy: ...done in 0.00 s
sfepy: nls: iter: 0, residual: 1.147589e+02 (rel: 1.000000e+00)
sfepy:   residual:    0.00 [s]
sfepy:     matrix:    0.00 [s]
sfepy:      solve:    0.01 [s]
sfepy: nls: iter: 1, residual: 1.649892e-13 (rel: 1.437703e-15)
sfepy: solved in 1 steps in 0.02 seconds


In [102]:
print('Nonlinear solver status:\n', nls_status)

Nonlinear solver status:
 IndexedStruct
  condition:
    0
  err:
    1.649891865843052e-13
  err0:
    114.75885522852423
  ls_n_iter:
    -1
  n_iter:
    1
  time_stats:
    dict with keys: ['residual', 'matrix', 'solve']


In [103]:
print('Stationary solver status:\n', status)

Stationary solver status:
 IndexedStruct
  n_step:
    1
  time:
    0.020894200000043384


In [104]:
pb.save_state('linear_elasticity.vtk', vec)

In [105]:
from sfepy.postprocess.viewer import Viewer

In [106]:
view = Viewer('regions.vtk')

In [107]:
view()

sfepy: reading mesh (regions.vtk)...
sfepy:   number of vertices: 258
sfepy:   number of cells:
sfepy:     2_3: 454
sfepy: ...done in 0.00 s
sfepy: point scalars Gamma1 at [-16. -10.   0.]
sfepy: range: 0.00e+00 1.00e+00 l2 norm range: 0.00e+00 1.00e+00
sfepy: point scalars Gamma2 at [ -5. -10.   0.]
sfepy: range: 0.00e+00 1.00e+00 l2 norm range: 0.00e+00 1.00e+00
sfepy: point scalars Omega at [  6. -10.   0.]
sfepy: range: 1.00e+00 1.00e+00 l2 norm range: 1.00e+00 1.00e+00


In [108]:
view = Viewer('linear_elasticity.vtk')

In [69]:
view()

sfepy: reading mesh (linear_elasticity.vtk)...
sfepy:   number of vertices: 258
sfepy:   number of cells:
sfepy:     2_3: 454
sfepy: ...done in 0.00 s
sfepy: point vectors u at [ -5. -10.   0.]
sfepy: range: -4.75e-01 1.00e+00 l2 norm range: 0.00e+00 1.11e+00


In [110]:
view(vector_mode='warp_norm', rel_scaling=1,is_scalar_bar=True, is_wireframe=True)

sfepy: reading mesh (linear_elasticity.vtk)...
sfepy:   number of vertices: 258
sfepy:   number of cells:
sfepy:     2_3: 454
sfepy: ...done in 0.00 s
sfepy: point vectors u at [ -5. -10.   0.]
sfepy: range: -1.00e+01 1.00e+01 l2 norm range: 0.00e+00 1.19e+01


In [70]:
import numpy as nm
from sfepy.discrete.fem import Mesh, FEDomain, Field